# Módulo 4 - Big Data

Estudiante: Jairo Prado

# Lectura y limpieza del dataset

In [0]:
# Cargar el conjunto de datos completo. Este paso no realiza ningún ajuste; simplemente lectura
#import findspark
#findspark.init('/usr/lib/python3.7/site-packages/pyspark')

from pyspark.sql.types import (StringType, IntegerType, FloatType, 
                               DecimalType, StructField, StructType, DoubleType)

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "postgresql-42.2.14.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.2.14.jar") \
    .getOrCreate()
#nota cambiar path en tarea como el ejemplo del profe 
diabetes_df = spark \
    .read \
    .format("csv") \
    .option("path", "diabetes_dataset.csv") \
    .option("header", True) \
    .schema(StructType([
                StructField("age", IntegerType()),
                StructField("gender", StringType()),
                StructField("ethnicity", StringType()),
                StructField("education_level", StringType()),
                StructField("income_level", StringType()),
                StructField("employment_status", StringType()),
                StructField("smoking_status", StringType()),
                StructField("alcohol_consumption_per_week",  IntegerType()),
                StructField("physical_activity_minutes_per_week", IntegerType()),
                StructField("diet_score", DoubleType()),
                StructField("sleep_hours_per_day", DoubleType()),
                StructField("screen_time_hours_per_day", DoubleType()),
                StructField("family_history_diabetes", IntegerType()),
                StructField("hypertension_history", IntegerType()),
                StructField("cardiovascular_history", IntegerType()),
                StructField("bmi", DoubleType()),
                StructField("waist_to_hip_ratio", DoubleType()),
                StructField("systolic_bp", IntegerType()),
                StructField("diastolic_bp", IntegerType()),
                StructField("heart_rate", IntegerType()),
                StructField("cholesterol_total", IntegerType()),
                StructField("hdl_cholesterol", IntegerType()),
                StructField("ldl_cholesterol", IntegerType()),
                StructField("triglycerides", IntegerType()),
                StructField("glucose_fasting", IntegerType()),
                StructField("glucose_postprandial", IntegerType()),
                StructField("insulin_level", DoubleType()),
                StructField("hba1c", DoubleType()),
                StructField("diabetes_risk_score", DoubleType()),
                StructField("diabetes_stage", StringType()),
                StructField("diagnosed_diabetes", IntegerType())])) \
    .load()

#diabetes_df.printSchema()
diabetes_df.display()

In [0]:
# Seleccionar un subconjunto de las columnas de interés y el objectivo que es el precio
columns_kept = ['age','gender','ethnicity','education_level','income_level','employment_status','smoking_status','alcohol_consumption_per_week','physical_activity_minutes_per_week','diet_score','sleep_hours_per_day','screen_time_hours_per_day','family_history_diabetes','hypertension_history','cardiovascular_history','bmi','waist_to_hip_ratio','systolic_bp','diastolic_bp','heart_rate','cholesterol_total','hdl_cholesterol','ldl_cholesterol','triglycerides','glucose_fasting','glucose_postprandial','insulin_level','hba1c']



selected_columns_df = houses_df.select(columns_kept)

selected_columns_df.show()

selected_columns_df.show()

In [0]:
# La cantidad de pisos y el precio fueron cargados como string. Deben ajustarse.
correct_types_df = selected_columns_df \
    .withColumn('floors', selected_columns_df['floors'].cast(FloatType())) \
    .withColumn('price', selected_columns_df['price'].cast(DecimalType()))


correct_types_df.show()

In [0]:
# Almacenar el conjunto de datos limpio en la base de datos
correct_types_df \
    .write \
    .format("jdbc") \
    .mode('overwrite') \
    .option("url", "jdbc:postgresql://host.docker.internal:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "houses") \
    .save()

# Inspección de datos
Previo a entrenar el modelo es común que se realice algún tipo de descripción de los datos, para tener una idea del tipo de problema con el que nos enfrentamos. A continuación, algunas operacions útiles que podemos usar.


In [0]:
# Cargar el conjunto de datos. Esta vez desde la base de datos

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "houses") \
    .load()

df.show()

In [0]:
# Información descriptiva del dataframe.
df.describe(['bedrooms', 'floors', 'yr_built']).show()

In [0]:
# Para realizar operaciones más detalladas es necesario expresar las filas originales en vectores
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
               'floors', 'condition', 'yr_built'],
    outputCol='features')

vector_df = assembler.transform(df)
vector_df = vector_df.select(['features', 'price'])
vector_df.show()


In [0]:
# Con la representación de vectores podemos calcular correlaciones
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt

pearson_matrix = Correlation.corr(vector_df, 'features').collect()[0][0]

sns.heatmap(pearson_matrix.toArray(), annot=True, fmt=".2f", cmap='viridis')

# Estandarización
Como recordamos de los módulos anteriores es deseable que los datos se encuentren estandarizados o normalizados, para evitar que la magnitud de ciertos atributos dominen el proceso de entrenamiento. El módulo `pyspark.ml.feature` tiene varias alternativas. A continuación se presenta una.

In [0]:
from pyspark.ml.feature import StandardScaler

standard_scaler = StandardScaler(inputCol='features', outputCol='scaled')
scale_model = standard_scaler.fit(vector_df)

scaled_df = scale_model.transform(vector_df)
scaled_df.show()

# Regresión lineal
Como ejemplo de un tipo de modelo a entrenar usaremos una regresión lineal básica para tratar de predecir el precio. Para ellos utilizaremos los datos estandarizados.

In [0]:
# Nótese que no se hace partición de datos de entrenamiento (ejercicio posterior).
from pyspark.ml.regression import LinearRegression

regression = LinearRegression(featuresCol='scaled', labelCol='price')
regression_model = regression.fit(scaled_df)

print('Pesos: {}\n b: {}'.format(regression_model.coefficients, regression_model.intercept))

print('RMSE: {} r2: {}'.format(
    regression_model.summary.rootMeanSquaredError,
    regression_model.summary.r2))

scaled_df.describe().show()

# Si se quiere hacer predicciones nada más se requiere llamar a regression_model.transform(df)
# Esto regresa el dataset con una columna nueva

# Ejercicio
- Dividir los datos leídos desde la base de datos en dos conjuntos utilizando las funcionalidades de muestreo de Spark.
- Entrene el modelo en uno y evalúe en otro. Escriba los resultados evaluados en una tabla en Postgresql.
- Implementar un esquema de k-fold cross validation. Evalúe los resultados según RMSE y r2.
- Pruebe diferentes parámetros a la regresión.
- Pruebe diferentes tipos de regresión.